In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions

In [ ]:
ks_2016=pd.read_csv('ks_2016.csv')

In [ ]:
import random

In [ ]:
#크롤링 block 당했거나 오류가 났을 때, 
#continue하고 다음 url로 바로 넘어가는게 아니라 약 2분 정도 간격으로 10번까지 접속시도 
#접속되면 scrape하고 안 되면 continue되도록 한다
options = ChromeOptions()

driver = Chrome(options=options)
content_list = []
for idx, url in enumerate( ks_2016['url_project'][7000:8000], start=7000):
    driver.get(url)
    time.sleep(random.uniform(8,20))
    req = driver.page_source
    soup = BeautifulSoup(req, 'lxml')
    try:
        content_tag = soup.select_one('div.rte__content')
        contents = content_tag.select('p')
        contents_collected = []
        for c in contents:
            content = c.get_text().strip()
            contents_collected.append(content)
        try:
            risk_challenge_tag = soup.select_one('div#risksAndChallenges')
            risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
            for rc in risk_challenge_list:
                risk_challenge = rc.get_text().strip()
        except:
            risk_challenge=" " #또는 np.nan
        #오류 나기 전까지 저장되어 있는 content_list에는 이어서 append
        content_list.append([idx, contents_collected, risk_challenge])
        #오류 난 후부턴 다시 크롤링 할 때, 매 페이지 긁어온 것마다 바로바로 csv파일에 한줄씩 append 할 것이다
        content_list_since_error = []
        content_list_since_error.append([idx, contents_collected, risk_challenge])
        pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv('ks_content_crawled_2016_7.csv', index=False, header=False, mode='a', encoding='UTF-8')
        print(idx)
    except Exception as ex:
        print("Unexpected error at {}".format(idx), ex)
        error_url = url
        count = 0
        while count < 11:
            print("Retry:", count)
            driver.get(error_url)
            time.sleep(random.uniform(8,20))
            req = driver.page_source
            soup = BeautifulSoup(req, 'lxml')
            try:
                content_tag = soup.select_one('div.rte__content')
                contents = content_tag.select('p')
                contents_collected = []
                for c in contents:
                    content = c.get_text().strip()
                    contents_collected.append(content)
                try:
                    risk_challenge_tag = soup.select_one('div#risksAndChallenges')
                    risk_challenge_list = risk_challenge_tag.select('p.js-risks-text.text-preline')
                    for rc in risk_challenge_list:
                        risk_challenge = rc.get_text().strip()
                except:
                    risk_challenge=" " #또는 np.nan
                content_list.append([idx, contents_collected, risk_challenge])
                content_list_since_error = []
                content_list_since_error.append([idx, contents_collected, risk_challenge])
                pd.DataFrame(content_list_since_error, columns=['index','content','risk_challenge']).to_csv('ks_content_crawled_2016_7.csv', index=False, header=False, mode='a', encoding='UTF-8')
                print(idx)
                break
            except Exception as et:
                print("Error while retrying:", et)
                time.sleep(random.uniform(100,130))
                count += 1
                continue
        #만약 재시도를 10번 했는데도 크롤링에 실패시 
        #오류 index, url, nan값을 csv파일들에 저장
        if count == 11:
            error_list = []
            error_list.append([idx, error_url])
            pd.DataFrame(error_list, columns=['index','error_url']).to_csv('ks_middle_errors_2016_7.csv', index=False, header=False, mode='a', encoding='UTF-8')
            content_list.append([idx, error_url, np.nan])
            error_into_saved_csv = []
            error_into_saved_csv.append([idx, error_url, np.nan])
            pd.DataFrame(error_into_saved_csv, columns=['index','content','risk_challenge']).to_csv('ks_content_crawled_2016_7.csv', index=False, header=False, mode='a', encoding='UTF-8')
        continue
driver.close()